### Import Packages

In [1]:
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)

### Read Dataset

In [2]:
species = 'Sardina pilchardus'
speciesName = 'Sardina Pilchardus'
# panel data
data_panel = pd.read_csv('T:/ownCloud/ODYSSEA/Aquamaps/the16species_final/'+species+'.csv',index_col='Unnamed: 0')
# fill missing values with interpolation
data_panel = data_panel.interpolate()
print(data_panel.shape[0], 'rows')

71520 rows


In [3]:
data_panel.head()

,id,Genus,Species,Center Lat,Center Long,C-Square Code,Overall Probability,obs_id,temperatureSurface,year_month,temperature100_300,temperature300_400,temperature100_500,temperatureMaxDepth,salinitySurface,salinity100_300,salinity300_400,salinity100_500,salinityMaxDepth,dissolvedOxygenSurface,dissolvedOxygen100_300,dissolvedOxygen300_400,dissolvedOxygen100_500,dissolvedOxygenMaxDepth,meridionalCurrentSurface,meridionalCurrent100_300,meridionalCurrent300_400,meridionalCurrent100_500,meridionalCurrentMaxDepth,zonalCurrentSurface,zonalCurrent100_300,zonalCurrent300_400,zonalCurrent100_500,zonalCurrentMaxDepth,chlorophyll,euphoticDepth,secchiDiskDepth,wave_Height,nitrateSurface,nitrate100_300,nitrate300_400,nitrate100_500,nitrateMaxDepth,phosphateSurface,phosphate100_300,phosphate300_400,phosphate100_500,phosphateMaxDepth,distanceToCoast,majorRiverDistance,majorRiversScale,bathymetry,substrateType,substrateOrigHabitat,substrateBiozone
0,02008-01,Sardina,pilchardus,30.75,17.75,1301:207:4,0.12,0,17.487333,2008-01,17.483358,17.483358,17.483358,17.483358,38.589620,38.589687,38.589687,38.589687,38.589687,215.00977,215.00972,215.00972,215.00972,215.00972,-0.004633,-0.004417,-0.004417,-0.004417,-0.004417,0.003990,0.003799,0.003799,0.003799,0.003799,0.091120,-0.787063,16.542532,0.364236,0.006429,0.006428,0.006428,0.006428,0.006428,0.003623,0.003623,0.003623,0.003623,0.003623,18.893757,1105.000196,8,-16.010401,Sand,A5.46: Mediterranean biocoenosis of coastal de...,Circalittoral
1,02008-02,Sardina,pilchardus,30.75,17.75,1301:207:4,0.12,0,16.356483,2008-02,16.323410,16.323410,16.323410,16.323410,38.357500,38.356500,38.356500,38.356500,38.356500,222.06847,222.06845,222.06845,222.06845,222.06845,0.000000,-0.004417,-0.004417,-0.004417,0.000000,0.000000,0.003799,0.003799,0.003799,0.000000,0.058834,-0.595206,14.983622,0.371151,0.005919,0.005919,0.005919,0.005919,0.005919,0.003703,0.003703,0.003703,0.003703,0.003703,18.893757,1105.000196,8,-16.010401,Sand,A5.46: Mediterranean biocoenosis of coastal de...,Circalittoral
2,02008-03,Sardina,pilchardus,30.75,17.75,1301:207:4,0.12,0,17.412617,2008-03,17.323942,17.323942,17.323942,17.323942,38.330997,38.329420,38.329420,38.329420,38.329420,222.28883,222.28891,222.28891,222.28891,222.28891,-0.018918,-0.017599,-0.017599,-0.017599,-0.017599,0.014905,0.015852,0.015852,0.015852,0.015852,0.088339,-0.858202,17.274120,0.173578,0.006872,0.006872,0.006872,0.006872,0.006872,0.003937,0.003937,0.003937,0.003937,0.003937,18.893757,1105.000196,8,-16.010401,Sand,A5.46: Mediterranean biocoenosis of coastal de...,Circalittoral
3,02008-04,Sardina,pilchardus,30.75,17.75,1301:207:4,0.12,0,18.664886,2008-04,18.580479,18.580479,18.580479,18.580479,38.356586,38.354680,38.354680,38.354680,38.354680,217.55254,217.55266,217.55266,217.55266,217.55266,-0.015815,-0.014742,-0.014742,-0.014742,-0.014742,0.014309,0.012531,0.012531,0.012531,0.012531,0.219484,-0.786224,20.733776,0.777014,0.006976,0.006975,0.006975,0.006975,0.006975,0.004803,0.004803,0.004803,0.004803,0.004803,18.893757,1105.000196,8,-16.010401,Sand,A5.46: Mediterranean biocoenosis of coastal de...,Circalittoral
4,02008-05,Sardina,pilchardus,30.75,17.75,1301:207:4,0.12,0,20.683325,2008-05,20.466887,20.466887,20.466887,20.466887,38.414630,38.409603,38.409603,38.409603,38.409603,211.78080,212.11305,212.11305,212.11305,212.11305,-0.010033,-0.009926,-0.009926,-0.009926,-0.009926,0.009827,0.007829,0.007829,0.007829,0.007829,0.079540,-0.866522,21.826303,0.296160,0.007796,0.007209,0.007209,0.007209,0.007209,0.004959,0.004966,0.004966,0.004966,0.004966,18.893757,1105.000196,8,-16.010401,Sand,A5.46: Mediterranean biocoenosis of coastal de...,Circalittoral


In [4]:
### Create some lists that will be useful
static_cols = ['distanceToCoast','majorRiverDistance', 'majorRiversScale', 'bathymetry', 'substrateType','substrateOrigHabitat', 'substrateBiozone']
temporal_cols = ['temperatureSurface','temperature100_300', 'temperature300_400', 'temperature100_500','temperatureMaxDepth', 'salinitySurface', 'salinity100_300','salinity300_400', 'salinity100_500', 'salinityMaxDepth','dissolvedOxygenSurface', 'dissolvedOxygen100_300','dissolvedOxygen300_400', 'dissolvedOxygen100_500','dissolvedOxygenMaxDepth', 'meridionalCurrentSurface','meridionalCurrent100_300', 'meridionalCurrent300_400','meridionalCurrent100_500', 'meridionalCurrentMaxDepth',
                 'zonalCurrentSurface', 'zonalCurrent100_300', 'zonalCurrent300_400','zonalCurrent100_500', 'zonalCurrentMaxDepth', 'chlorophyll','euphoticDepth', 'secchiDiskDepth', 'wave_Height', 'nitrateSurface','nitrate100_300', 'nitrate300_400', 'nitrate100_500', 'nitrateMaxDepth','phosphateSurface', 'phosphate100_300', 'phosphate300_400','phosphate100_500', 'phosphateMaxDepth']

### Create a new dataset with the mean value of each observation. This is the mean of 120 months.
data = pd.DataFrame()
for c in data_panel.columns:
    if c in temporal_cols:
        data[c+'_mean'] = data_panel.groupby('obs_id')[c].mean()
        data[c+'_std'] = data_panel.groupby('obs_id')[c].std()
        data[c+'_min'] = data_panel.groupby('obs_id')[c].min()
        data[c+'_max'] = data_panel.groupby('obs_id')[c].max()
    if c in static_cols:
        data[c] = data_panel.groupby('obs_id')[c].agg(lambda x:x.value_counts().index[0])
    if c in ['Center Lat','Center Long','Overall Probability']:
        data[c] = data_panel.groupby('obs_id')[c].agg(lambda x:x.value_counts().index[0])

In [5]:
print(data.shape[0], 'rows and ', data.shape[1], 'columns')
data.head()

596 rows and  166 columns


,Center Lat,Center Long,Overall Probability,temperatureSurface_mean,temperatureSurface_std,temperatureSurface_min,temperatureSurface_max,temperature100_300_mean,temperature100_300_std,temperature100_300_min,temperature100_300_max,temperature300_400_mean,temperature300_400_std,temperature300_400_min,temperature300_400_max,temperature100_500_mean,temperature100_500_std,temperature100_500_min,temperature100_500_max,temperatureMaxDepth_mean,temperatureMaxDepth_std,temperatureMaxDepth_min,temperatureMaxDepth_max,salinitySurface_mean,salinitySurface_std,salinitySurface_min,salinitySurface_max,salinity100_300_mean,salinity100_300_std,salinity100_300_min,salinity100_300_max,salinity300_400_mean,salinity300_400_std,salinity300_400_min,salinity300_400_max,salinity100_500_mean,salinity100_500_std,salinity100_500_min,salinity100_500_max,salinityMaxDepth_mean,salinityMaxDepth_std,salinityMaxDepth_min,salinityMaxDepth_max,dissolvedOxygenSurface_mean,dissolvedOxygenSurface_std,dissolvedOxygenSurface_min,dissolvedOxygenSurface_max,dissolvedOxygen100_300_mean,dissolvedOxygen100_300_std,dissolvedOxygen100_300_min,dissolvedOxygen100_300_max,dissolvedOxygen300_400_mean,dissolvedOxygen300_400_std,dissolvedOxygen300_400_min,dissolvedOxygen300_400_max,dissolvedOxygen100_500_mean,dissolvedOxygen100_500_std,dissolvedOxygen100_500_min,dissolvedOxygen100_500_max,dissolvedOxygenMaxDepth_mean,dissolvedOxygenMaxDepth_std,dissolvedOxygenMaxDepth_min,dissolvedOxygenMaxDepth_max,meridionalCurrentSurface_mean,meridionalCurrentSurface_std,meridionalCurrentSurface_min,meridionalCurrentSurface_max,meridionalCurrent100_300_mean,meridionalCurrent100_300_std,meridionalCurrent100_300_min,meridionalCurrent100_300_max,meridionalCurrent300_400_mean,meridionalCurrent300_400_std,meridionalCurrent300_400_min,meridionalCurrent300_400_max,meridionalCurrent100_500_mean,meridionalCurrent100_500_std,meridionalCurrent100_500_min,meridionalCurrent100_500_max,meridionalCurrentMaxDepth_mean,meridionalCurrentMaxDepth_std,meridionalCurrentMaxDepth_min,meridionalCurrentMaxDepth_max,zonalCurrentSurface_mean,zonalCurrentSurface_std,zonalCurrentSurface_min,zonalCurrentSurface_max,zonalCurrent100_300_mean,zonalCurrent100_300_std,zonalCurrent100_300_min,zonalCurrent100_300_max,zonalCurrent300_400_mean,zonalCurrent300_400_std,zonalCurrent300_400_min,zonalCurrent300_400_max,zonalCurrent100_500_mean,zonalCurrent100_500_std,zonalCurrent100_500_min,zonalCurrent100_500_max,zonalCurrentMaxDepth_mean,zonalCurrentMaxDepth_std,zonalCurrentMaxDepth_min,zonalCurrentMaxDepth_max,chlorophyll_mean,chlorophyll_std,chlorophyll_min,chlorophyll_max,euphoticDepth_mean,euphoticDepth_std,euphoticDepth_min,euphoticDepth_max,secchiDiskDepth_mean,secchiDiskDepth_std,secchiDiskDepth_min,secchiDiskDepth_max,wave_Height_mean,wave_Height_std,wave_Height_min,wave_Height_max,nitrateSurface_mean,nitrateSurface_std,nitrateSurface_min,nitrateSurface_max,nitrate100_300_mean,nitrate100_300_std,nitrate100_300_min,nitrate100_300_max,nitrate300_400_mean,nitrate300_400_std,nitrate300_400_min,nitrate300_400_max,nitrate100_500_mean,nitrate100_500_std,nitrate100_500_min,nitrate100_500_max,nitrateMaxDepth_mean,nitrateMaxDepth_std,nitrateMaxDepth_min,nitrateMaxDepth_max,phosphateSurface_mean,phosphateSurface_std,phosphateSurface_min,phosphateSurface_max,phosphate100_300_mean,phosphate100_300_std,phosphate100_300_min,phosphate100_300_max,phosphate300_400_mean,phosphate300_400_std,phosphate300_400_min,phosphate300_400_max,phosphate100_500_mean,phosphate100_500_std,phosphate100_500_min,phosphate100_500_max,phosphateMaxDepth_mean,phosphateMaxDepth_std,phosphateMaxDepth_min,phosphateMaxDepth_max,distanceToCoast,majorRiverDistance,majorRiversScale,bathymetry,substrateType,substrateOrigHabitat,substrateBiozone
obs_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,30.75,17.75,0.12,22.373630,4.222837,16.102207,29.556072,22.

### Encode Categorical Features

In [6]:
for c in data:
    if (data[c].dtype=='object'):
        lbl = LabelEncoder() 
        lbl.fit(list(data[c].values))
        data[c] = lbl.transform(list(data[c].values))

## Train Models

In [7]:
ntrain = data.shape[0]
SEED = 2019 # for reproducibility
NFOLDS = 5
# Define Cross Validation
folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

# Define evaluation function (Root Mean Square Error)
def cv_rmse(model, X, y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=folds))
    return (rmse)

In [8]:
cols_to_exclude = ['obs_id','Overall Probability']
df_train_columns = [c for c in data.columns if c not in cols_to_exclude]

y_train = data['Overall Probability'].ravel() #ravel coverts a series to a numpy array
x_train = data[df_train_columns].values # converts a dataframe to a numpy array

### LightGBM

In [9]:
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )

score = cv_rmse(lightgbm, x_train, y_train)
print("lightgbm: {:.4f} ({:.4f})".format(score.mean(), score.std()) )

lightgbm: 0.1377 (0.0206)
